In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=9yk1doMydlsaFPrTQsbQ4D0tQSW8i4-dbBafvCTzoFU&tc=eUNAPqgFO12a1ZURkUN-VBKvz3FztGiElDdeKl7l774&cc=a_V20U7I9ayIxP2bT24nGd9i70ci3Tlk7JaygFqP2Vc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh6JfgoMSijW517IA33QT62mlt5CozKXMg-VxxZ0wo3RwKJQ1Kzonpw

Successfully saved authorization token.


In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import folium

In [ ]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()}).set({'DOY':img.date().getRelative('day', 'year')})
  return reduce_region_function

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [ ]:
country_name = 'Nigeria'
longitude = 12.697000
latitude = 9.097900

point = ee.Geometry.Point(longitude, latitude)
geom = point.buffer(distance= 20000)

country_name = country_name.replace(' ','')
print('country : ', country_name)

country :  Nigeria


In [ ]:
precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").sort('system:time_start').filterBounds(geom)

In [ ]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-20, 'years'), today)
precipitation = precipitation.filterDate(date_range).select('precipitation')

In [ ]:
precipitation.size().getInfo()

7268

In [ ]:
precipitation.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'precipitation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7200, 2000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 50]}],
 'version': 1515007194560647,
 'id': 'UCSB-CHG/CHIRPS/DAILY/20030310',
 'properties': {'system:time_start': 1047254400000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1047340800000,
  'system:asset_size': 3953603,
  'system:index': '20030310'}}

In [ ]:
def Avg_8(img):
  date = ee.Date(ee.List(img.get('system:time_start')))
  date_range = ee.DateRange(date.advance(-8, 'days'), date)
  imagecollection = precipitation.filterDate(date_range)
  return imagecollection.mean().set({'millis': img.date().millis()}).set({'system:time_start':img.get('system:time_start')}).set({'DOY':img.date().getRelative('day', 'year')})

In [ ]:
avg_data = precipitation.map(Avg_8)

In [ ]:
avg_data = avg_data.filter(ee.Filter.Or(
    ee.Filter.eq('DOY', 0),
    ee.Filter.eq('DOY', 8),
    ee.Filter.eq('DOY', 16),
    ee.Filter.eq('DOY', 24),
    ee.Filter.eq('DOY', 32),
    ee.Filter.eq('DOY', 40),
    ee.Filter.eq('DOY', 48),
    ee.Filter.eq('DOY', 56),
    ee.Filter.eq('DOY', 64),
    ee.Filter.eq('DOY', 72),
    ee.Filter.eq('DOY', 80),
    ee.Filter.eq('DOY', 88),
    ee.Filter.eq('DOY', 96),
    ee.Filter.eq('DOY', 104),
    ee.Filter.eq('DOY', 112),
    ee.Filter.eq('DOY', 120),
    ee.Filter.eq('DOY', 128),
    ee.Filter.eq('DOY', 136),
    ee.Filter.eq('DOY', 144),
    ee.Filter.eq('DOY', 152),
    ee.Filter.eq('DOY', 160),
    ee.Filter.eq('DOY', 168),
    ee.Filter.eq('DOY', 176),
    ee.Filter.eq('DOY', 184),
    ee.Filter.eq('DOY', 192),
    ee.Filter.eq('DOY', 200),
    ee.Filter.eq('DOY', 208),
    ee.Filter.eq('DOY', 216),
    ee.Filter.eq('DOY', 224),
    ee.Filter.eq('DOY', 232),
    ee.Filter.eq('DOY', 240),
    ee.Filter.eq('DOY', 248),
    ee.Filter.eq('DOY', 256),
    ee.Filter.eq('DOY', 264),
    ee.Filter.eq('DOY', 272),
    ee.Filter.eq('DOY', 280),
    ee.Filter.eq('DOY', 288),
    ee.Filter.eq('DOY', 296),
    ee.Filter.eq('DOY', 304),
    ee.Filter.eq('DOY', 312),
    ee.Filter.eq('DOY', 320),
    ee.Filter.eq('DOY', 328),
    ee.Filter.eq('DOY', 336),
    ee.Filter.eq('DOY', 344),
    ee.Filter.eq('DOY', 352),
    ee.Filter.eq('DOY', 360)
))

In [ ]:
reduce_precipitation = create_reduce_region_function(
    geometry=geom, reducer=ee.Reducer.mean(), scale=250, crs='EPSG:4326')

Precipitation_stat_fc = ee.FeatureCollection(avg_data.map(reduce_precipitation)).filter(ee.Filter.notNull(avg_data.first().bandNames()))

In [ ]:
asset_path = 'projects/project_name/' + str(country_name) + '_Precipitation_fc_ts_vis_with_altair_buffering20km'

task = ee.batch.Export.table.toAsset(
    collection=Precipitation_stat_fc,
    description='Precipitation_stat_fc export',
    assetId=asset_path)

task.start()   # It takes some time to make a data

In [ ]:
precipitation_stat_fc = ee.FeatureCollection(asset_path)

In [ ]:
precipitation_dict = fc_to_dict(precipitation_stat_fc).getInfo()

In [ ]:
print(type(precipitation_dict), '\n')
for prop in precipitation_dict.keys():
    print(prop + ':', precipitation_dict[prop][0:3] + ['...'])

<class 'dict'> 

DOY: [0, 0, 0, '...']
millis: [1293840000000, 1546300800000, 1640995200000, '...']
precipitation: [0.0002336669373294419, 0.0017503940541076367, 0.0002557519382903458, '...']
system:index: ['00000000000000000167', '000000000000000002d7', '00000000000000000361', '...']


In [ ]:
precipitation_df = pd.DataFrame(precipitation_dict)
display(precipitation_df)
print(precipitation_df.dtypes)

,DOY,millis,precipitation,system:index
0,0,1293840000000,0.000234,00000000000000000167
1,0,1546300800000,0.001750,000000000000000002d7
2,0,1640995200000,0.000256,00000000000000000361
3,0,1072915200000,0.000000,00000000000000000025
4,0,1104537600000,0.000000,00000000000000000053
...,...,...,...,...
910,248,1536192000000,8.345019,000000000000000002c8
911,248,1567728000000,12.191252,000000000000000002f6
912,248,1599264000000,9.589130,00000000000000000324
913,248,1630886400000,4.187695,00000000000000000352


DOY                int64
millis             int64
precipitation    float64
system:index      object
dtype: object


In [ ]:
# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  return df

In [ ]:
precipitation_df = add_date_info(precipitation_df)
precipitation_df.head(5)

,DOY,millis,precipitation,system:index,Timestamp,Year,Month,Day
0,0,1293840000000,0.000234,00000000000000000167,2011-01-01,2011,1,1
1,0,1546300800000,0.001750,000000000000000002d7,2019-01-01,2019,1,1
2,0,1640995200000,0.000256,00000000000000000361,2022-01-01,2022,1,1
3,0,1072915200000,0.000000,00000000000000000025,2004-01-01,2004,1,1
4,0,1104537600000,0.000000,00000000000000000053,2005-01-01,2005,1,1


In [ ]:
precipitation_df = precipitation_df.drop(columns=['millis', 'system:index'])
precipitation_df = precipitation_df.sort_values(by='Timestamp').reset_index(drop=True)
precipitation_df.head(5)

,DOY,precipitation,Timestamp,Year,Month,Day
0,72,0.001324,2003-03-14,2003,3,14
1,80,0.000141,2003-03-22,2003,3,22
2,88,0.087886,2003-03-30,2003,3,30
3,96,1.019198,2003-04-07,2003,4,7
4,104,3.811269,2003-04-15,2003,4,15


In [ ]:
precipitation_df.dtypes

DOY                       int64
precipitation           float64
Timestamp        datetime64[ns]
Year                      int64
Month                     int64
Day                       int64
dtype: object

In [ ]:
alt.Chart(precipitation_df).mark_bar(size=1).encode(
    x='Timestamp:T',
    y='precipitation:Q',
    color=alt.Color(
        'precipitation:Q', scale=alt.Scale(scheme='turbo', domain=(precipitation_df.precipitation.min(),precipitation_df.precipitation.max()))),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('precipitation:Q', title='precipitation')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True, bind='legend')
color = alt.condition(highlight,
                      alt.Color('Year:O', title='Year'),
                      alt.value('lightgray'))

base = alt.Chart(precipitation_df).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 353], clamp=True)),
    y=alt.Y('precipitation:Q', scale=alt.Scale(domain=[precipitation_df.precipitation.min(),precipitation_df.precipitation.max()])),
    color=color)

points = base.mark_circle().encode(
    opacity=alt.value(0),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Timestamp:T', title='DOY'),
        alt.Tooltip('precipitation:Q', title='precipitation')
    ]).add_selection(highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)))

(points + lines).properties(width=1200, height=600)

alt.LayerChart(...)

In [ ]:
series_dropdown = alt.binding_select(options=sorted(set(precipitation_df["Year"])))
series_selection = alt.selection_single(
    fields=["Year"], bind=series_dropdown, name="_"
)

line_chart = (
    alt.Chart(precipitation_df)
    .mark_line()
    .encode(
      x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 353], clamp=True)),
      y=alt.Y('precipitation:Q', scale=alt.Scale(domain=[precipitation_df.precipitation.min(),precipitation_df.precipitation.max()])),
        detail=alt.Detail(field="Year",type="nominal"),
        color=alt.condition(
            predicate=series_selection,
            if_true=alt.value("steelblue"),
            if_false=alt.value("lightgray"),
        ),
        opacity=alt.condition(
            predicate=series_selection,
            if_true=alt.value(1),
            if_false=alt.value(0.5),
        ),
    )
)


line_chart.add_selection(series_selection).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
mean_DOY = precipitation_df.groupby('DOY').mean()[['precipitation']]
mean_DOY.head()

,precipitation
DOY,
0,0.000112
8,0.000713
16,0.000383
24,0.000926
32,0.000760


In [ ]:
mean_DOY[mean_DOY.index == 0]['precipitation'][0]

0.00011199064648637121

In [ ]:
difference_list = []
for i in range(len(precipitation_df)):
  DOY = precipitation_df['DOY'][i]
  precipitation_value = precipitation_df['precipitation'][i]
  difference = precipitation_value - mean_DOY[mean_DOY.index == DOY]['precipitation'][DOY]
  difference_list.append(difference)

In [ ]:
precipitation_df['difference'] = difference_list

In [ ]:
precipitation_df.head()

,DOY,precipitation,Timestamp,Year,Month,Day,difference
0,72,0.001324,2003-03-14,2003,3,14,-0.044235
1,80,0.000141,2003-03-22,2003,3,22,-0.049116
2,88,0.087886,2003-03-30,2003,3,30,-0.264434
3,96,1.019198,2003-04-07,2003,4,7,0.509818
4,104,3.811269,2003-04-15,2003,4,15,2.232259


In [ ]:
alt.Chart(precipitation_df).mark_bar(size=2).encode(
    x='Timestamp:T',
    y='difference:Q',
    color=alt.condition(
        predicate=alt.datum.difference >= 0,
        if_true=alt.value("blue"),
        if_false=alt.value("red")
    ),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('difference:Q', title='Difference')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
precipitation_df = precipitation_df.sort_values(by='Timestamp').reset_index(drop=True)

In [ ]:
precipitation_df.iloc[12:,:].shape

(903, 7)

In [ ]:
average_precipitation = []
average_difference = []

for i in range(12,len(precipitation_df)):
  df = precipitation_df.iloc[i-12:i,:]
  average_precipitation.append(df.precipitation.mean())
  average_difference.append(df.difference.mean())

In [ ]:
len(average_precipitation)

903

In [ ]:
precipitation_df2 = precipitation_df.iloc[12:,:].copy()
precipitation_df2['average_precipitation'] = average_precipitation
precipitation_df2['average_difference'] = average_difference

In [ ]:
alt.Chart(precipitation_df2).mark_bar(size=3).encode(
    x='Timestamp:T',
    y='average_precipitation:Q',
    color=alt.Color(
        'average_precipitation:Q', scale=alt.Scale(scheme='turbo', domain=(precipitation_df2.average_precipitation.min(),precipitation_df2.average_precipitation.max()))),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('average_precipitation:Q', title='average_precipitation')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
alt.Chart(precipitation_df2).mark_bar(size=2).encode(
    x='Timestamp:T',
    y='average_difference:Q',
    color=alt.condition(
        predicate=alt.datum.average_difference >= 0,
        if_true=alt.value("blue"),
        if_false=alt.value("red")
    ),
    tooltip=[
        alt.Tooltip('Timestamp:T', title='Date'),
        alt.Tooltip('average_difference:Q', title='average_difference')
    ]).properties(width=1200, height=600)

alt.Chart(...)

In [ ]:
outputpath = '/content/drive/MyDrive/Precipitation_buffering20km/' + country_name+ '.csv'
precipitation_df2.to_csv(outputpath, index=None)